# Preguntas y respuestas 

# ENTREGABLE 3

## Nota! Ejecutar primera celda para disponibilizar datos de los csv que se usan en las posteriores consultas


In [ ]:
import pandas as pd
sales_df = pd.read_csv(r'data/sales.csv')
products_df = pd.read_csv(r'data/products.csv')
employees_df = pd.read_csv(r'data/employees.csv')
categories_df=pd.read_csv(r'data/categories.csv')
customers_df = pd.read_csv(r'data/customers.csv')

## Columna con datos inválidos
__Contexto__
El campo TotalPrice en la tabla sales no contiene valores válidos, por lo que necesitamos calcular el valor real de la venta utilizando la información de precios disponible en la tabla products. La fórmula a utilizar es:

TotalPriceCalculated = (Quantity × UnitPrice) × (1 − Discount)

* Donde:
* * Quantity es la cantidad de productos vendidos.
  * UnitPrice es el precio unitario del producto desde la tabla products.
  * Discount es el descuento aplicado al producto (rango de 0 a 1).

__Análisis preliminar__
- a) Estructura de la tabla


In [3]:
import pandas as pd
import utils.pandas_utils as p_utils

p_utils.get_dataframe_info(sales_df)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6758125 entries, 0 to 6758124
Data columns (total 9 columns):
 #   Column             Dtype  
---  ------             -----  
 0   SalesID            int64  
 1   SalesPersonID      int64  
 2   CustomerID         int64  
 3   ProductID          int64  
 4   Quantity           int64  
 5   Discount           float64
 6   TotalPrice         float64
 7   SalesDate          object 
 8   TransactionNumber  object 
dtypes: float64(2), int64(5), object(2)
memory usage: 464.0+ MB



* b) Verificación de los registros inválidos para el campo **TotalPrice**.
Para comenzar, se realizó una inspección preliminar para identificar los registros con valor igual o menor a cero (dado que no tiene sentido que un valor de venta tome esos valores).


In [6]:
import utils.notebook_utils as notebook_utils
import utils.pandas_utils as p_utils

sum_zero_or_negative = p_utils.sum_zero_or_negative(sales_df, 'TotalPrice')
notebook_utils.print_colored(f'Total rows on TotalPrice (zero or negative): {sum_zero_or_negative}', 'orange')   


* c) Verificación de la validez de los valores en **Discount**.

La fórmula de cálculo de TotalPrice requiere que Discount esté dentro del rango esperado [0, 1]. Se realizó una verificación para encontrar registros donde el descuento no esté en el rango adecuado:

In [7]:
import utils.notebook_utils as notebook_utils

rows_out_in_range = sales_df['Discount'].between(0, 1, inclusive='both')
notebook_utils.print_colored(f'Total rows on Discount (between 0 and 1): {len(rows_out_in_range)}', 'orange')

__Conclusiones__

* b) Todos los registros en TotalPrice son cero o negativos.
* c) Todos los registros en Discount están dentro del rango esperado [0, 1].


__ACCIONES__

* Se procede a calcular precio total de venta, con la fórmula propuesta y agregando dicho resultado en una nueva columna   : **AggregatedTotalPrice**

In [3]:
sales_data= sales_df.copy()
products_data = products_df.copy()

sales_with_price = sales_data.merge(products_data[['ProductID', 'Price']], on='ProductID', how='left') # agrega el precio del producto

sales_with_price['AggregatedTotalPrice'] = sales_with_price['Quantity'] * sales_with_price['Price'] * (1 - sales_with_price['Discount'])

sales_with_price.head()

,SalesID,SalesPersonID,CustomerID,ProductID,Quantity,Discount,TotalPrice,SalesDate,TransactionNumber,Price,AggregatedTotalPrice
0,1,6,27039,381,7,0.0,0.0,2018-02-05 07:38:25.430,FQL4S94E4ME1EZFTG42G,44.2337,309.63590
1,2,16,25011,61,7,0.0,0.0,2018-02-02 16:03:31.150,12UGLX40DJ1A5DTFBHB8,62.5460,437.82200
2,3,13,94024,23,24,0.0,0.0,2018-05-03 19:31:56.880,5DT8RCPL87KI5EORO7B0,79.0184,1896.44160
3,4,8,73966,176,19,0.2,0.0,2018-04-07 14:43:55.420,R3DR9MLD5NR76VO17ULE,81.3167,1236.01384
4,5,10,32653,310,9,0.0,0.0,2018-02-12 15:37:03.940,4BGS0Z5OMAZ8NDAFHHP3,79.9780,719.80200


__Comentario__

Tras el ajuste y calcular la nueva columna AggregatedTotalPrice, se verificó que existen algunas filas con valores menores o iguales a cero. Esto se debe a la presencia de dos productos cuyo precio es cero en la tabla products.

Por el momento, se ha decidido conservar estos registros, ya que el alcance de la actividad ha sido cumplido, y en sí para evaluar las ventas totales es un porcentaje infimo (<0,45%) sin embargo se deja registro porque refiere a dos productos particulares .

In [10]:
sum_zero_or_negative = p_utils.sum_zero_or_negative(sales_with_price, 'AggregatedTotalPrice')
notebook_utils.print_colored(f'Total rows on AggregatedTotalPrice (zero or negative): {sum_zero_or_negative}', 'orange') 

zero_or_negative_products = products_data[products_data['Price'] <= 0]
notebook_utils.print_colored(f'Total rows on products with zero or negative price: {len(zero_or_negative_products)} ', 'orange')

zero_or_negative_products

,ProductID,ProductName,Price,CategoryID,Class,ModifyDate,Resistant,IsAllergic,VitalityDays
18,19,Tea - Earl Grey,0.0,3,Medium,39:53.3,Weak,TRUE,94
154,155,Peas - Pigeon; Dry,0.0,9,High,39:50.1,Weak,FALSE,0


## Detectar los outliers en la columna de ventas totales (TotalPriceCalculated)

__Contexto__

Utilizando el criterio del rango intercuartílico (IQR). Luego, crea una nueva columna llamada IsOutlier que tenga el valor 1 si el registro es un outlier y 0 en caso contrario. ¿Cuántos outliers se detectaron?

__ACCIONES__

Se calcularon cuartiles y se contabilizó cantidad de registros fuera de los límites calculados

In [4]:
import utils.notebook_utils as notebook_utils
import utils.pandas_utils as p_utils
q1 = sales_with_price['AggregatedTotalPrice'].quantile(0.25)
q2 = sales_with_price['AggregatedTotalPrice'].quantile(0.5)
q3 = sales_with_price['AggregatedTotalPrice'].quantile(0.75)

iqr = q3 - q1
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr
sales_data = sales_with_price.copy()


sales_data['isOutlier'] = p_utils.is_outlier(sales_data, 'AggregatedTotalPrice', lower_bound, upper_bound)

notebook_utils.print_colored(f"Total rows with outliers: {sales_data['isOutlier'].sum()}", 'orange')
sales_data.head()


,SalesID,SalesPersonID,CustomerID,ProductID,Quantity,Discount,TotalPrice,SalesDate,TransactionNumber,Price,AggregatedTotalPrice,isOutlier
0,1,6,27039,381,7,0.0,0.0,2018-02-05 07:38:25.430,FQL4S94E4ME1EZFTG42G,44.2337,309.63590,0
1,2,16,25011,61,7,0.0,0.0,2018-02-02 16:03:31.150,12UGLX40DJ1A5DTFBHB8,62.5460,437.82200,0
2,3,13,94024,23,24,0.0,0.0,2018-05-03 19:31:56.880,5DT8RCPL87KI5EORO7B0,79.0184,1896.44160,0
3,4,8,73966,176,19,0.2,0.0,2018-04-07 14:43:55.420,R3DR9MLD5NR76VO17ULE,81.3167,1236.01384,0
4,5,10,32653,310,9,0.0,0.0,2018-02-12 15:37:03.940,4BGS0Z5OMAZ8NDAFHHP3,79.9780,719.80200,0


__Conclusiones__ 
Aunque la dispersión de los datos es considerable (IQR = 805), resulta razonable dado que se trata de un negocio con una amplia variedad de productos y categorías, lo que implica distintos rangos de precios.

La proporción de registros detectados como outliers considerando el criterio de rango intercuartílico (IQR) es muy baja (<1%), con apenas 48,217 casos sobre un total de 6,758,125. Esto sugiere que no hay una presencia significativa de valores extremos que justifique su eliminación.

Para futuros análisis, se sugiere re-evaluar la dispersión y los outliers excluyendo productos cuyo precio unitario sea igual a cero (ya identificados), estos podrían estar afectando artificialmente la variabilidad del conjunto de datos. Es posible que esta depuración reduzca tanto el IQR como el número de outliers detectados, mejorando la calidad de los análisis posteriores.

## A partir de la columna SalesDate, crea una nueva columna que contenga únicamente la hora de la venta.

__Objetivos__ 
Luego, identifica en qué hora del día se concentran más ventas totales (TotalPriceCalculated).

¿La empresa vende más durante los días de semana o en el fin de semana? Utiliza la columna SalesDate para identificar el día de la semana de cada venta, clasifica los registros como Entre semana o Fin de semana, y compara el total de ventas (TotalPriceCalculated) entre ambos grupos.

__Acciones__

Se da formato a columna **SalesDate** para parsearla de string a tipo datetime y se agrega el resultado en la columna SalesTime

In [6]:
import utils.notebook_utils as notebook_utils
import utils.pandas_utils as p_utils

sales_data = sales_with_price.copy()

sales_data['SalesDate'] = pd.to_datetime(sales_data['SalesDate'], errors='coerce')
sales_data['SalesTime'] = sales_data['SalesDate'].dt.hour

sales_data.head()


,SalesID,SalesPersonID,CustomerID,ProductID,Quantity,Discount,TotalPrice,SalesDate,TransactionNumber,Price,AggregatedTotalPrice,SalesTime
0,1,6,27039,381,7,0.0,0.0,2018-02-05 07:38:25.430,FQL4S94E4ME1EZFTG42G,44.2337,309.63590,7.0
1,2,16,25011,61,7,0.0,0.0,2018-02-02 16:03:31.150,12UGLX40DJ1A5DTFBHB8,62.5460,437.82200,16.0
2,3,13,94024,23,24,0.0,0.0,2018-05-03 19:31:56.880,5DT8RCPL87KI5EORO7B0,79.0184,1896.44160,19.0
3,4,8,73966,176,19,0.2,0.0,2018-04-07 14:43:55.420,R3DR9MLD5NR76VO17ULE,81.3167,1236.01384,14.0
4,5,10,32653,310,9,0.0,0.0,2018-02-12 15:37:03.940,4BGS0Z5OMAZ8NDAFHHP3,79.9780,719.80200,15.0


Se evalua en que horario se concentran las ventas totales, agrupando por hora y buscando el maximo de tales agrupamientos. 

__Resultado__

La hora del día con mayor concentración de ventas reportado es la hora 16 .

In [9]:
sales_by_hour = sales_data.groupby('SalesTime')['AggregatedTotalPrice'].sum().reset_index()
max_sales_hour = sales_by_hour.loc[sales_by_hour['AggregatedTotalPrice'].idxmax()]

notebook_utils.print_colored(f"The hour with the highest sales is {max_sales_hour['SalesTime']} with a total of {max_sales_hour['AggregatedTotalPrice']:.2f}.", 'orange')

__Análisis__
Se evalua si hay mayor volumen de ventas en días de semana o en fin de semana. 
Para esto se agrupan las ventas en dos grupos `weekday`y `weekend` y se evalúa total de ventas para cada grupo.

In [7]:
sales_data['SalesDay'] = sales_data['SalesDate'].dt.day_name()
sales_by_day = sales_data['SalesDay'].apply(lambda x: 'Weekday' if x in ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'] else 'Weekend')
sales_by_day_group = sales_data.groupby(sales_by_day)['AggregatedTotalPrice'].sum().reset_index()

notebook_utils.print_colored(f'Total sales by day group:', 'orange')

sales_by_day_group.head()

,SalesDay,AggregatedTotalPrice
0,Weekday,3.080352e+09
1,Weekend,1.235916e+09


__Conclusion__ 
En términos absolutos, se observa un mayor volumen de ventas durante los días de semana. Sin embargo, esto puede estar relacionado a que hay más días laborables que en fin de semana (cinco contra dos).
Para una comparación más justa, se calculó el promedio diario de ventas en cada grupo:
Promedio entre semana: 616.070.400
Promedio fin de semana: 617.958.000

Aunque la diferencia no es significativa, el fin de semana presenta un promedio diario de ventas ligeramente superior. Esto sugiere un mejor rendimiento individual de ventas en esos días.



## Como parte del proceso de feature engineering, en el mismo df que vienes trabajando, calcula dos nuevas columnas en el dataset de ventas:

* La edad del empleado al momento de su contratación y años de experiencia al momento de realizar cada venta.

__Acciones__

* Agregar los datos a la tabla de ventas con AggregatedTotalPrice.
* * En primer lugar se convierten a tipo datetime las fechas de nacimiento y contratación.
* * Luego se agregan columnas en la tabla de trabajo

In [8]:
employees_df['BirthDate'] = pd.to_datetime(employees_df['BirthDate'], errors='coerce')
employees_df['HireDate'] = pd.to_datetime(employees_df['HireDate'], errors='coerce')

sales_dataframe=sales_with_price.merge(employees_df[['EmployeeID', 'BirthDate', 'HireDate']], left_on='SalesPersonID', right_on='EmployeeID' ,how='left')
sales_dataframe.head()

sales_dataframe['EmployeeAgeAtHire'] = (sales_dataframe['HireDate'] - sales_dataframe['BirthDate']).dt.days // 365
sales_dataframe['EmployeeAgeAtHire'] = sales_dataframe['EmployeeAgeAtHire'].fillna(0).astype(int)
sales_dataframe['EmployeeCareerYears'] = (pd.to_datetime('today') - sales_dataframe['HireDate']).dt.days // 365
sales_dataframe.head()

,SalesID,SalesPersonID,CustomerID,ProductID,Quantity,Discount,TotalPrice,SalesDate,TransactionNumber,Price,AggregatedTotalPrice,EmployeeID,BirthDate,HireDate,EmployeeAgeAtHire,EmployeeCareerYears
0,1,6,27039,381,7,0.0,0.0,2018-02-05 07:38:25.430,FQL4S94E4ME1EZFTG42G,44.2337,309.63590,6,1987-01-13,2013-06-22 13:20:18.080,26,11
1,2,16,25011,61,7,0.0,0.0,2018-02-02 16:03:31.150,12UGLX40DJ1A5DTFBHB8,62.5460,437.82200,16,1951-07-07,2017-02-10 11:21:26.650,65,8
2,3,13,94024,23,24,0.0,0.0,2018-05-03 19:31:56.880,5DT8RCPL87KI5EORO7B0,79.0184,1896.44160,13,1963-04-18,2011-12-12 10:43:52.940,48,13
3,4,8,73966,176,19,0.2,0.0,2018-04-07 14:43:55.420,R3DR9MLD5NR76VO17ULE,81.3167,1236.01384,8,1956-12-13,2014-10-14 23:12:53.420,57,10
4,5,10,32653,310,9,0.0,0.0,2018-02-12 15:37:03.940,4BGS0Z5OMAZ8NDAFHHP3,79.9780,719.80200,10,1963-12-30,2012-07-23 15:02:12.640,48,12


## CIERRE

Preparar un único dataset definitivo para modelado que combine información relevante de las tablas disponibles.

Incluir las features que se han calculado previamente.

Aplicar transformaciones adecuadas a las variables categóricas y a las variables numéricas (si lo consideras necesario) para dejar los datos listos para ser utilizados por un modelo de machine learning.

Justifica las transformaciones realizadas. La variable objetivo es TotalPriceCalculated, por lo que debe quedar sin transformaciones.



__ACCIONES__

1) Se agregaron features ya calculadas y se detalla su utilidad:
    - isOutlier: La inclusión de esta columna es relevante porque ayuda a identificar y manejar los registros que se desvían significativamente de las tendencias normales de ventas.
    - SalesDate a datetime: La conversión de SalesDate a tipo datetime es ncesaria para facilitar la extracción de componentes de la fecha, como la hora, el día de la semana, etc.
    - SalesTime, SalesDay y SalesDayGroup:: Para el modelo es significativo conocer tendencias en ventas como son horas y días de mayor rendimiento.


In [9]:

sales_dataframe['isOutlier'] = p_utils.is_outlier(sales_dataframe, 'AggregatedTotalPrice', lower_bound, upper_bound)
sales_dataframe['SalesDate'] = pd.to_datetime(sales_dataframe['SalesDate'], errors='coerce')
sales_dataframe['SalesTime'] = sales_dataframe['SalesDate'].dt.hour
sales_dataframe['SalesDay'] = sales_dataframe['SalesDate'].dt.day_name()
sales_dataframe['SalesDayGroup'] = sales_dataframe['SalesDay'].apply(lambda x: 'Weekday' if x in ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'] else 'Weekend')

sales_dataframe.head()

,SalesID,SalesPersonID,CustomerID,ProductID,Quantity,Discount,TotalPrice,SalesDate,TransactionNumber,Price,AggregatedTotalPrice,EmployeeID,BirthDate,HireDate,EmployeeAgeAtHire,EmployeeCareerYears,isOutlier,SalesTime,SalesDay,SalesDayGroup
0,1,6,27039,381,7,0.0,0.0,2018-02-05 07:38:25.430,FQL4S94E4ME1EZFTG42G,44.2337,309.63590,6,1987-01-13,2013-06-22 13:20:18.080,26,11,0,7.0,Monday,Weekday
1,2,16,25011,61,7,0.0,0.0,2018-02-02 16:03:31.150,12UGLX40DJ1A5DTFBHB8,62.5460,437.82200,16,1951-07-07,2017-02-10 11:21:26.650,65,8,0,16.0,Friday,Weekday
2,3,13,94024,23,24,0.0,0.0,2018-05-03 19:31:56.880,5DT8RCPL87KI5EORO7B0,79.0184,1896.44160,13,1963-04-18,2011-12-12 10:43:52.940,48,13,0,19.0,Thursday,Weekday
3,4,8,73966,176,19,0.2,0.0,2018-04-07 14:43:55.420,R3DR9MLD5NR76VO17ULE,81.3167,1236.01384,8,1956-12-13,2014-10-14 23:12:53.420,57,10,0,14.0,Saturday,Weekend
4,5,10,32653,310,9,0.0,0.0,2018-02-12 15:37:03.940,4BGS0Z5OMAZ8NDAFHHP3,79.9780,719.80200,10,1963-12-30,2012-07-23 15:02:12.640,48,12,0,15.0,Monday,Weekday


2) Se agregaron columnas de otras tablas
- Tabla **products**
    - IsAllergic >>> Si indica si el producto puede generar alergias, puede influir en en el comportamiento de compra.	
    - VitalityDays >>> Es una variable numérica que podría relacionarse con la duración o utilidad del producto; puede ser relevante para asociar con volumen de ventas, rotación.

- Tabla **categories**
    - CategoryName: Proporciona el nombre de la categoría a la que pertenece cada producto. Es una variable categórica relevante, especialmente para segmentar y comparar entre categorías.

- Tabla **customers**
    - CityID: Este dato puede ser útil para segmentar geográficamente.



In [10]:

sales_dataframe = sales_dataframe.merge(products_df[['ProductID', 'CategoryID', 'IsAllergic', 'VitalityDays']], on='ProductID', how='left')
sales_dataframe = sales_dataframe.merge(categories_df[['CategoryID', 'CategoryName']], on='CategoryID', how='left')
sales_dataframe = sales_dataframe.merge(customers_df[['CustomerID', 'CityID']], on='CustomerID', how='left')


sales_dataframe.head()

,SalesID,SalesPersonID,CustomerID,ProductID,Quantity,Discount,TotalPrice,SalesDate,TransactionNumber,Price,...,EmployeeCareerYears,isOutlier,SalesTime,SalesDay,SalesDayGroup,CategoryID,IsAllergic,VitalityDays,CategoryName,CityID
0,1,6,27039,381,7,0.0,0.0,2018-02-05 07:38:25.430,FQL4S94E4ME1EZFTG42G,44.2337,...,11,0,7.0,Monday,Weekday,1.0,Unknown,41.0,Confections,54
1,2,16,25011,61,7,0.0,0.0,2018-02-02 16:03:31.150,12UGLX40DJ1A5DTFBHB8,62.5460,...,8,0,16.0,Friday,Weekday,8.0,FALSE,90.0,Grain,71
2,3,13,94024,23,24,0.0,0.0,2018-05-03 19:31:56.880,5DT8RCPL87KI5EORO7B0,79.0184,...,13,0,19.0,Thursday,Weekday,11.0,TRUE,0.0,Produce,2
3,4,8,73966,176,19,0.2,0.0,2018-04-07 14:43:55.420,R3DR9MLD5NR76VO17ULE,81.3167,...,10,0,14.0,Saturday,Weekend,6.0,TRUE,90.0,Seafood,45
4,5,10,32653,310,9,0.0,0.0,2018-02-12 15:37:03.940,4BGS0Z5OMAZ8NDAFHHP3,79.9780,...,12,0,15.0,Monday,Weekday,9.0,FALSE,0.0,Poultry,82


3) LIMPIEZA 

    a- Se eliminaron columnas irrelevantes de cara a analizar venta total. Detalle de columnas a eliminar y justificación:

* TotalPrice: todas sus filas tienen valor cero y fue reemplazada por dato enriquecido **AggregatedTotalPrice**.

* TransactionNumber: irrelevante para el análisis, es un id de identificacion.

* BirthDate: reemplazada por su derivada **EmployeeAgeAtHire**.

* HireDate y SalesDate: son de tipo object y ya fueron descompuestas en variables más útiles para el modelo (EmployeeCareerYears, SalesTime, SalesDay, SalesDayGroup). Admás ,la eliminacón ayuda a simplificar los datos, manteniendo solo las variables relevantes.


In [11]:
sales_dataframe = sales_dataframe.drop(columns=['TotalPrice','TransactionNumber', 'BirthDate', 'EmployeeAgeAtHire', 'HireDate', 'SalesDate'])

sales_dataframe.head()

,SalesID,SalesPersonID,CustomerID,ProductID,Quantity,Discount,Price,AggregatedTotalPrice,EmployeeID,EmployeeCareerYears,isOutlier,SalesTime,SalesDay,SalesDayGroup,CategoryID,IsAllergic,VitalityDays,CategoryName,CityID
0,1,6,27039,381,7,0.0,44.2337,309.63590,6,11,0,7.0,Monday,Weekday,1.0,Unknown,41.0,Confections,54
1,2,16,25011,61,7,0.0,62.5460,437.82200,16,8,0,16.0,Friday,Weekday,8.0,FALSE,90.0,Grain,71
2,3,13,94024,23,24,0.0,79.0184,1896.44160,13,13,0,19.0,Thursday,Weekday,11.0,TRUE,0.0,Produce,2
3,4,8,73966,176,19,0.2,81.3167,1236.01384,8,10,0,14.0,Saturday,Weekend,6.0,TRUE,90.0,Seafood,45
4,5,10,32653,310,9,0.0,79.9780,719.80200,10,12,0,15.0,Monday,Weekday,9.0,FALSE,0.0,Poultry,82


3) LIMPIEZA 

    b- Verificar duplicidad entre SalesPersonId y EmployeeID

In [19]:
(sales_dataframe['SalesPersonID'] == sales_dataframe['EmployeeID']).all()

np.True_

-> Como son iguales se elimina SalesPersonID para evitar redundancia. 

In [12]:
sales_dataframe = sales_dataframe.drop(columns=['SalesPersonID'])
sales_dataframe.head()

,SalesID,CustomerID,ProductID,Quantity,Discount,Price,AggregatedTotalPrice,EmployeeID,EmployeeCareerYears,isOutlier,SalesTime,SalesDay,SalesDayGroup,CategoryID,IsAllergic,VitalityDays,CategoryName,CityID
0,1,27039,381,7,0.0,44.2337,309.63590,6,11,0,7.0,Monday,Weekday,1.0,Unknown,41.0,Confections,54
1,2,25011,61,7,0.0,62.5460,437.82200,16,8,0,16.0,Friday,Weekday,8.0,FALSE,90.0,Grain,71
2,3,94024,23,24,0.0,79.0184,1896.44160,13,13,0,19.0,Thursday,Weekday,11.0,TRUE,0.0,Produce,2
3,4,73966,176,19,0.2,81.3167,1236.01384,8,10,0,14.0,Saturday,Weekend,6.0,TRUE,90.0,Seafood,45
4,5,32653,310,9,0.0,79.9780,719.80200,10,12,0,15.0,Monday,Weekday,9.0,FALSE,0.0,Poultry,82


4) TRANFORMACIONES

    a- IsAllergic
        -  toma valores True, False, unknown 
        - se evalua frcuencia de unknown para definir aplicar binarización o one hot encoding.

In [13]:

unknown_count = (sales_dataframe['IsAllergic'] == 'Unknown').sum()
notebook_utils.print_colored(f"Total 'Unknown' values in 'IsAllergic': {unknown_count}", 'orange')


__Resultado__: 
- Se encontró que el valor Unknown representa aproximadamente un tercio del total de los registros, lo que lo hace relevante y no debe ser ignorado.
- Se aplicó One-Hot Encoding a la columna IsAllergic, creando tres columnas binarias:



In [16]:
sales_dataframe = pd.get_dummies(sales_dataframe, columns=['IsAllergic'], prefix='IsAllergic')

cols = [col for col in sales_dataframe.columns if col.startswith('IsAllergic_')]
sales_dataframe[cols] = sales_dataframe[cols].astype(int)

sales_dataframe.head()


,SalesID,CustomerID,ProductID,Quantity,Discount,Price,AggregatedTotalPrice,EmployeeID,EmployeeCareerYears,isOutlier,SalesTime,SalesDay,SalesDayGroup,CategoryID,VitalityDays,CategoryName,CityID,IsAllergic_FALSE,IsAllergic_TRUE,IsAllergic_Unknown
0,1,27039,381,7,0.0,44.2337,309.63590,6,11,0,7.0,Monday,Weekday,1.0,41.0,Confections,54,0,0,1
1,2,25011,61,7,0.0,62.5460,437.82200,16,8,0,16.0,Friday,Weekday,8.0,90.0,Grain,71,1,0,0
2,3,94024,23,24,0.0,79.0184,1896.44160,13,13,0,19.0,Thursday,Weekday,11.0,0.0,Produce,2,0,1,0
3,4,73966,176,19,0.2,81.3167,1236.01384,8,10,0,14.0,Saturday,Weekend,6.0,90.0,Seafood,45,0,1,0
4,5,32653,310,9,0.0,79.9780,719.80200,10,12,0,15.0,Monday,Weekday,9.0,0.0,Poultry,82,1,0,0


4) TRANFORMACIONES

    b- Discount (binarización de la columna **Discount**)
    - Se agregó nueva columna HadDiscount para indicar si una venta tuvo o no descuento, el objetivo es simplificar análisis abstrayendo el modelo de los valores específicos del descuento. 

In [18]:

sales_dataframe['HadDiscount'] = (sales_dataframe['Discount'] > 0).astype(int)

sales_dataframe.head()


,SalesID,CustomerID,ProductID,Quantity,Discount,Price,AggregatedTotalPrice,EmployeeID,EmployeeCareerYears,isOutlier,...,SalesDay,SalesDayGroup,CategoryID,VitalityDays,CategoryName,CityID,IsAllergic_FALSE,IsAllergic_TRUE,IsAllergic_Unknown,HadDiscount
0,1,27039,381,7,0.0,44.2337,309.63590,6,11,0,...,Monday,Weekday,1.0,41.0,Confections,54,0,0,1,0
1,2,25011,61,7,0.0,62.5460,437.82200,16,8,0,...,Friday,Weekday,8.0,90.0,Grain,71,1,0,0,0
2,3,94024,23,24,0.0,79.0184,1896.44160,13,13,0,...,Thursday,Weekday,11.0,0.0,Produce,2,0,1,0,0
3,4,73966,176,19,0.2,81.3167,1236.01384,8,10,0,...,Saturday,Weekend,6.0,90.0,Seafood,45,0,1,0,1
4,5,32653,310,9,0.0,79.9780,719.80200,10,12,0,...,Monday,Weekday,9.0,0.0,Poultry,82,1,0,0,0


4) TRANFORMACIONES

    b- SalesDayGroup (binarización)
    - Se agregó columna onWeekendSale para convertir la variable de categorica a binaria.


In [20]:

sales_dataframe['onWeekendSale'] = sales_dataframe['SalesDayGroup'].map({
    'Weekday': 0,
    'Weekend': 1
})
sales_dataframe.head()

,SalesID,CustomerID,ProductID,Quantity,Discount,Price,AggregatedTotalPrice,EmployeeID,EmployeeCareerYears,isOutlier,...,SalesDayGroup,CategoryID,VitalityDays,CategoryName,CityID,IsAllergic_FALSE,IsAllergic_TRUE,IsAllergic_Unknown,HadDiscount,onWeekendSale
0,1,27039,381,7,0.0,44.2337,309.63590,6,11,0,...,Weekday,1.0,41.0,Confections,54,0,0,1,0,0
1,2,25011,61,7,0.0,62.5460,437.82200,16,8,0,...,Weekday,8.0,90.0,Grain,71,1,0,0,0,0
2,3,94024,23,24,0.0,79.0184,1896.44160,13,13,0,...,Weekday,11.0,0.0,Produce,2,0,1,0,0,0
3,4,73966,176,19,0.2,81.3167,1236.01384,8,10,0,...,Weekend,6.0,90.0,Seafood,45,0,1,0,1,1
4,5,32653,310,9,0.0,79.9780,719.80200,10,12,0,...,Weekday,9.0,0.0,Poultry,82,1,0,0,0,0


4) TRANFORMACIONES

    c - CategoriesName
           

In [31]:
sales_dataframe["CategoryName"].describe()

count         6758108
unique             11
top       Confections
freq           851976
Name: CategoryName, dtype: object

- Análisis:

* Hay 6,758,108 registros no nulos en la columna CategoryName. Esto significa que la mayoría de las filas en tu conjunto de datos tienen un valor asignado en esta columna.

* 11 categorías únicas.
* "Confections" aparece 851,976 veces en la columna, es la categoría más abundante en ventas .

Dado que CategoryName tiene un número moderado de categorías (11), One-Hot Encoding puede ser una estrategia adecuada, por dos motivos : por un lado se desconoce modelo ML a usar y por otra parte las caegorías no tienen un orden o jerarquía entre sí, co lo cual es una opción agnóstica en tales sentidos 

In [22]:

sales_dataframe = pd.get_dummies(sales_dataframe, columns=['CategoryName'], prefix='Category')
cols = [col for col in sales_dataframe.columns if col.startswith('Category_')]
sales_dataframe[cols] = sales_dataframe[cols].astype(int)

sales_dataframe.head()

,SalesID,CustomerID,ProductID,Quantity,Discount,Price,AggregatedTotalPrice,EmployeeID,EmployeeCareerYears,isOutlier,...,Category_Cereals,Category_Confections,Category_Dairy,Category_Grain,Category_Meat,Category_Poultry,Category_Produce,Category_Seafood,Category_Shell fish,Category_Snails
0,1,27039,381,7,0.0,44.2337,309.63590,6,11,0,...,0,1,0,0,0,0,0,0,0,0
1,2,25011,61,7,0.0,62.5460,437.82200,16,8,0,...,0,0,0,1,0,0,0,0,0,0
2,3,94024,23,24,0.0,79.0184,1896.44160,13,13,0,...,0,0,0,0,0,0,1,0,0,0
3,4,73966,176,19,0.2,81.3167,1236.01384,8,10,0,...,0,0,0,0,0,0,0,1,0,0
4,5,32653,310,9,0.0,79.9780,719.80200,10,12,0,...,0,0,0,0,0,1,0,0,0,0


5) OBSERVACIONES

    b-VitalityDays podría servir para entender rotacion o compras en gran volumen. Puede brindar información útil para ajustar precios, logística, stock.

In [27]:
sales_dataframe['VitalityDays'].describe()


count    6.758108e+06
mean     2.603534e+01
std      3.902604e+01
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      5.200000e+01
max      1.200000e+02
Name: VitalityDays, dtype: float64

- Análisis

Al revisar la columna VitalityDays, se observó que en algunas categorías, como Confections, los productos pueden ser tanto alimenticios como no alimenticios. Esta mezcla dificulta definir la idea de vida útil. Así un valor VitalityDays = 0 puede interpretarse como vencido/ expirado,pero para un producto no alimenticio el sentido es sencillamente la ausencia de fecha de vencimiento.

Se sugiere:

- Revisión y redefinición de las categorías para separar claramente productos alimenticios y no alimenticios.

- Lo anterior permitiría relacionar los valores posibles a las caracterisiticas de cada producto.Tratamiento de valores nulos para VitalityDays = 0 en productos alimenticios, ya sea mediante imputación con el promedio de la categoría o marcando esos valores como no confiables.


# Conclusión
Esta tercera entrega, basada en las dos primeras ,  implicó el procesamiento y análisis de un conjunto de datos de ventas de gran escala, con más de **8 millones de registros**. A lo largo de las distintas etapas, he profundizado en la comprensión del valor que tiene la ingeniería de datos para la toma de decisiones y la optimización de procesos de negocio.

El manejo de volúmenes masivos de información representó un desafío adicional, especialmente en entornos con recursos limitados. Para enfrentarlo, se aplicaron buenas prácticas de eficiencia, como la vectorización de operaciones (en lugar de bucles explícitos), y se utilizaron herramientas como pandas, que ofrecen una gestión eficiente de memoria y procesamiento.

La exploración y transformación de datos se documentaron y visualizaron principalmente mediante notebooks, lo que permitió una presentación clara y estructurada de resultados, facilitando tanto el análisis como la toma de decisiones sobre la marcha.

Por último, estas entregas también evidencian la importancia de comprender el negocio detrás de los datos. Sin ese entendimiento, el análisis corre el riesgo de basarse en suposiciones arbitrarias, limitando el aporte real a una solución concreta. La ingeniería de datos no solo transforma datos, sino que también requiere traducir necesidades del negocio en decisiones técnicas coherentes.